# Step 1:

In this step we carry out the following:

1. We load the spreadsheet from Sharepoint which includes the list of
   submissions, the human defined categories for submitters of interest and any
   further annotations such as whether we are removing submissions from
   analysis. We format this data into a json file, saved in
   `./data/step1/list.json`. This JSON file allows for easier manipulation and
   handling.
2. Using the JSON file from step 1, we create a number of jsonl files that are
   in the correct format for processing by OpenAI's batch API. We create
   multiple jsonl files as each has to be less than 100mb in size.
3. We upload the jsonl files, and trigger the batch processing of them. This can
   take upwards of 24 hours.
4. Once processing is complete, we download the completed responses for each
   request, and update the json file from step 1 to include the AI returned
   data. We also export this data in a spreadsheet for review
   (`./data/step1/review1.xlsx`)

After this step, not only do we have preliminary data for all submissions
(answers to questions below), but we can also check if the AI has catagorised
any of the unlabeled submissions (that we may have missed) into the categories
of interest. Once we are settled on the categories, we can move onto `Step 2`
which involves asking the AI the specific questions for each category.


## 1. Loading spreadsheet

[The spreadsheet](https://studentutsedu.sharepoint.com/:x:/r/sites/CentreforMediaTransition76/_layouts/15/doc2.aspx?sourcedoc=%7B26015E46-DC17-46F8-85CB-8FF7601BB93E%7D&file=List%20of%20all%20submissions.xlsx&action=default&mobileredirect=true&DefaultItemOpen=1&ct=1715733887026&wdOrigin=OFFICECOM-WEB.START.REC&cid=c80abce9-d7d3-419b-8b78-2504ae4ce71a&wdPreviousSessionSrc=HarmonyWeb&wdPreviousSession=a8d0354e-a231-454b-a8a5-18124a5f1983)


These are the files that the text was manually extracted for (as opposed to
being extracted by the Marker package)

```python
list_of_files_manually_text_extracted = ['e656', '14193', '18110', '19712', '26222', '33824', '34418', '34756', 'e597' ]
```


In [ ]:
# CELL 1
import json
import pandas as pd
import json
import os
from datetime import datetime
import pytz

# Load the Excel spreadsheet into a pandas DataFrame
df = pd.read_excel('./data/step1/list.xlsx')

# Convert the DataFrame to a list of dictionaries
data = df.to_dict(orient='records')

def extract_name_from_filename(filename):
    filename = filename.replace('acma2023-', '')
    parts = filename.split('-')
    name_parts = parts[1:]
    name = ' '.join(name_parts).split('.')[0]
    if name.find('anonymous') != -1:
        name = 'anonymous'
    return name.lower()

missing_files = []
manual_files = []

list_of_files_manually_text_extracted = ['e656', '14193', '18110', '19712', '26222', '33824', '34418', '34756', 'e597' ]

def check_file_exists(doc_id, folder_path = './data/files'):
    for file_name in os.listdir(folder_path):
        if file_name.startswith(doc_id) and doc_id not in list_of_files_manually_text_extracted:
            return True
    if doc_id not in list_of_files_manually_text_extracted:
        missing_files.append(doc_id)
        return False
    manual_files.append(doc_id)
    return False

formatted_data = []
# Convert empty cells in 'Group', 'Comments', and 'Removed (Y)' columns to None
for row in data:
    if pd.isnull(row['Group']):
        row['Group'] = None
    if pd.isnull(row['Comments']):
        row['Comments'] = None
    if pd.isnull(row['Removed (Y)']):
        row['Removed (Y)'] = None

    exist = check_file_exists(row['UniqueID'])
    
    formatted_row = {
        'uniqueId': row['UniqueID'],
        'group': row['Group'],
        'submitter': extract_name_from_filename(row['doc']),        
        "metadata": {
            "groupDefinedBy": "human" if row['Group'] else "AI",
            "removed": row['Removed (Y)'],
            "comments": row['Comments'],
            "text_extraction_method": "Marker2" if exist else 'manual'
        }
    }
    formatted_data.append(formatted_row)
    
# Save the data as a JSON file if it doesn't exist
json_file = './data/step1/list.json'

local_timezone = pytz.timezone('Australia/Sydney')  # Adjust to your local timezone
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

output = {
    "data": formatted_data,
    "metadata": {
        "excel_to_json": {
            "timestamp": current_time,
            "submissions_missing_files": missing_files,
            "submissions_using_manually_extracted_files": manual_files
        }
    }
}

with open(json_file, 'w') as f:
    json.dump(output, f)

print(F'{len(missing_files)} submissions missing files: {missing_files}')
print(F'{len(manual_files)} submissions using manually extracted files: {manual_files}')

## 2. We now have a JSON file of objects with key value pairs in the below form. We now process this to jsonl form for batch processing

```json
{
  "group": "string | null",
  "submitter": "string",
  "uniqueId": "string",
  "metadata": {
    "groupDefinedBy": "human or AI",
    "removed": "string | null",
    "comments": "string | null",
    "text_extraction_method": "Marker2 | manual"
  }
}
```

We ignore in this step any submissions we flag as removed.

jsonl files saved to `./data/step1/toProcess`


In [ ]:
# CELL 2
import os
import json
from datetime import datetime
import pytz

promt_file = 'prompt_no_guidance.txt'

# Define the prompt for each individual request
def prompt_formatted() -> str:    
    # Read the first file and set a string variable
    with open(promt_file, 'r') as file:
        prompt = file.read()
        
    with open('prompt_issues.md', 'r') as file:
        issues = file.read()

    # with open('prompt_guidance_note.md', 'r') as file:
    #     guidance_note = file.read()

    with open('prompt_fact_sheet.md', 'r') as file:
        fact_sheet = file.read()

    prompt = prompt.replace('|issues|', issues)
    # prompt = prompt.replace('|guidance_note|', guidance_note)
    prompt = prompt.replace('|fact_sheet|', fact_sheet)    

    return prompt

def get_submission(submission_text: str, submission_author: str):
    prompt = ""
    prompt += "\n\n***************************************** SUBMISSION START *****************************************\n\n"

    prompt += f"Submission from: {submission_author}\n\n"
    
    prompt += submission_text

    prompt += "\n\n***************************************** SUBMISSION END *****************************************\n\n"
    return prompt

def get_function():
    with open('function.json', 'r') as f:
        function = json.load(f)
    return function

def get_file_path(doc_id, folder_path = './data/files'):    
    for file_name in os.listdir(folder_path):
        if file_name.startswith(doc_id):
            return os.path.join(folder_path, file_name)

with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

md_file_location = './data/files'

file_counter = 0
jsonl_file = f"./data/step1/toProcess/jsonl_{file_counter}.jsonl"

skipped_files = []
empty_files = []

# Make step 1 directories
os.makedirs('./data/step1/toProcess', exist_ok=True)
os.makedirs('./data/step1/output', exist_ok=True)

prompt = prompt_formatted()

# This loop takes each submission and adds it to the jsonl file in a format that can be used by the OpenAI API
for i in list["data"]:   
    if i["metadata"]["removed"] == "Y":
        skipped_files.append(i["uniqueId"])
        continue
    try:
        md_file_path = get_file_path(i.get("uniqueId"))       
        with open(md_file_path, 'r') as file:
            submission = file.read()        
        if len(submission.strip()) == 0:
            i["metadata"]["removed"] = "Y"
            i["metadata"]["comments"] = f"{i["metadata"]["comments"]}\n\nRemoved due to empty file"
            empty_files.append(i["uniqueId"])
            continue
        sub_author = i["submitter"]
        submission_formatted = get_submission(submission, sub_author)
        function = get_function()

        ldata = { "custom_id": i["uniqueId"], "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-2024-05-13", "messages": [{"role": "system", "content": prompt},{"role": "user", "content": submission_formatted}], "max_tokens": 4096, "temperature": 1e-9, "frequency_penalty": 0, "presence_penalty": 0, "top_p": 0, "tools":[function], "tool_choice": { "type": "function", "function": { "name": "submission_eval" } }}}

        i["metadata"]["SUBMISSION_CONTENT"] =  submission_formatted
        
        if os.path.exists(jsonl_file) and os.path.getsize(jsonl_file) >= 85 * 1024 * 1024:  # 
            file_counter += 1
            jsonl_file = f"./data/step1/toProcess/jsonl_{file_counter}.jsonl"
        
        i["metadata"]["step_1"] = {"batch_file": f'jsonl_{file_counter}.jsonl'}
        
        with open(jsonl_file, 'a') as f:
            json.dump(ldata, f)
            f.write('\n')

    except Exception as e:
        print(e)
        continue

local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

list["metadata"]["step_1"] = { "jsonl_batch_creation": {
            "timestamp": current_time,
            "skipped_files": skipped_files,
            "empty_files": empty_files,
            "total_jsonL_files": file_counter + 1,
            }, 
            "ai_parameters" : { "custom_id": "SUBMISSION_ID", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-2024-05-13", "messages": [{"role": "system", "content": prompt},{"role": "user", "content": "SUBMISSION_CONTENT"}], "max_tokens": 4096, "temperature": 1e-9, "frequency_penalty": 0, "presence_penalty": 0, "top_p": 0, "tools":[function], "tool_choice": { "type": "function", "function": { "name": "submission_eval" } }}}
        }

with open('./data/step1/list.json', 'w') as f:
    json.dump(list, f)

print(f"Empty files: {empty_files}")

## 3. We now have a folder with all the prepared files for OpenAI batch calls

We will upload each of these files to OpenAI and then trigger batch processing
of each.

**MAKE SURE TO RECORD BATCH IDs CREATED IN THIS STEP SO WE KNOW WHICH FILES TO
EVENTUALLY DOWNLOAD**


In [9]:
# CELL 3
from openai import OpenAI
import os
from datetime import datetime
import pytz

client = OpenAI(api_key=os.getenv('OPENAI_KEY'), max_retries=3)

jsonl_dir = './data/step1/toProcess'

jsonl_files = [f for f in os.listdir(jsonl_dir) if os.path.isfile(os.path.join(jsonl_dir, f)) and f.endswith('.jsonl')]

file_ids = []

for file in jsonl_files:
    file_object = client.files.create(
        file=open(f"{jsonl_dir}/{file}", "rb"),
        purpose="batch"
    )
    file_ids.append(file_object.id)

with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

list["metadata"]["step_1"]["file_upload"] = {
            "timestamp": current_time,
            "file_ids": file_ids,            
        }

with open('./data/step1/list.json', 'w') as f:
    json.dump(list, f)

In [ ]:
# CELL 4
from openai import OpenAI
import os
from datetime import datetime
import pytz

client = OpenAI(api_key=os.getenv('OPENAI_KEY'), max_retries=3)

# We have now uploaded all the files and have their IDs, lets create a batch job for each
batch_ids = []

with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

file_ids = list["metadata"]["step_1"]["file_upload"]["file_ids"]

for file_id in file_ids:
    job = client.batches.create(
            input_file_id=file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
          )
    batch_ids.append(job.id)

print('Record the following and make sure to add to `desired_batch_ids` in the following cells!')
print(batch_ids)

local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

list["metadata"]["step_1"]["batch_creation"] = {
            "timestamp": current_time,
            "batch_ids": batch_ids,
        }

with open('./data/step1/list.json', 'w') as f:
    json.dump(list, f)

#### The batch processes should now be underway, they will take up to 24hrs

We can run the following cell to check on process


In [ ]:
from openai import OpenAI
import json

client = OpenAI(api_key=os.getenv('OPENAI_KEY'),max_retries=3)

with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

desired_batch_ids = list["metadata"]["step_1"]["batch_creation"]["batch_ids"]

batch_jobs = client.batches.list()

for batch in batch_jobs.data:
    if batch.id in desired_batch_ids:
        print(batch.id, batch.status, batch.request_counts)

#### Once processing is done, we can download the completed files

Files are saved here: `./data/step1/output`


In [24]:
# CELL 5
from openai import OpenAI
from datetime import datetime
import pytz

client = OpenAI(api_key=os.getenv('OPENAI_KEY'),max_retries=3)

batch_jobs = client.batches.list()

with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

# we only want to download the batch jobs that were set up in cell 11
desired_batch_ids = list["metadata"]["step_1"]["batch_creation"]["batch_ids"]

success_count = 0
error_count = 0

created_time = None
completion_time = None

for batch in batch_jobs.data:
    if batch.id in desired_batch_ids:
        # Gets the first created batch time
        if not created_time or batch.created_at < created_time:
            created_time = batch.created_at
        # Gets the lasted completed batch time
        if not completion_time or batch.completed_at > completion_time:
            completion_time = batch.completed_at
        success_count += batch.request_counts.total - batch.request_counts.failed
        output_file = batch.output_file_id
        content = client.files.content(output_file)
        jsonl_file_path = f'./data/step1/output/{output_file}.jsonl'
        content.write_to_file(jsonl_file_path)
        # Handle error files
        error_count += batch.request_counts.failed
        err_file = batch.error_file_id
        err_content = client.files.content(err_file)
        err_jsonl_file_path = f'./data/step1/output/err_{err_file}.jsonl'
        err_content.write_to_file(err_jsonl_file_path)

local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

time_difference = completion_time - created_time

list["metadata"]["step_1"]["batch_creation"]["download_timestamp"] = current_time
list["metadata"]["step_1"]["batch_creation"]["completetion_duration_seconds"] = time_difference
list["metadata"]["step_1"]["batch_creation"]["success"] = success_count
list["metadata"]["step_1"]["batch_creation"]["errors"] = error_count

with open('./data/step1/list.json', 'w') as f:
    json.dump(list, f)

## 4. Process AI responses and save data

Now we have all the AI responses, we need to process and save the results. This
will update the json file from step 2, and also export the responses as an Excel
file for review. The Excel file will be located: `./data/step1/review1/xlsx`


In [35]:
# CELL 6
from openai import OpenAI
import os
import json
import pandas as pd
from datetime import datetime
import pytz

client = OpenAI(api_key=os.getenv('OPENAI_KEY'),max_retries=3)

# Parses the JSON from a function call, if there is an error in JSON parsing, recalls the LLM with the fix json function to get a valid json response.
def parse_JSON(json_str: str) -> dict:        
    try: 
        return json.loads(json_str)
    except Exception as e:              
        messages = [
      {
        'role': 'system',
        'content':
          'Assistant is a large language model designed to fix and return correct JSON objects.',
      },
      {
        'role': 'user',
        'content': f'ORIGINAL ERROR CONTAINING JSON OBJECT:\n\n{json_str}\n\nERROR MESSAGE: {e}',
      },
    ]
        
        tool_choices = [{
      'type': 'function',
      'function': {
        'name': 'fix_object',
        'description':
          'You will be given an incorrectly formed JSON Object and a error message. You must fix the incorrect JSON Object and return the valid JSON object.',
        'parameters': {
          'type': 'object',
          'properties': {
            'fixedJSON': {
              'type': 'string',
              'description': 'The reformated and error free JSON object. Return the JSON object only!',
            },
          },
          'required': ['fixedJSON'],
        },
      },
    }]                
        response = client.chat.completions.create(
                    model='gpt-4o-2024-05-13',
                    messages=messages,                    
                    max_tokens=4096,
                    temperature=0,
                    tools=tool_choices,
                    tool_choice={ 'type': 'function', 'function': { 'name': 'fix_object' } },        
                )        
                
        second_test_json = response.choices[0].message.tool_calls[0].function.arguments 
                  
        to_return = json.loads(second_test_json)
        return json.loads(to_return['fixedJSON'])

output_folder = './data/step1/output'

jsonl_files = [f for f in os.listdir(output_folder) if os.path.isfile(os.path.join(output_folder, f)) and f.endswith('.jsonl')]

# Load original JSON list
with open('./data/step1/list.json', 'r') as f:
    list_data = json.load(f)

def get_correct_category(AI_category):
    AI_category = AI_category.lower()    
    if AI_category == 'digital platform':
        return 'platform'
    if AI_category == 'civil society':
        return 'civil'
    return AI_category

prompt_tokens = 0
completion_tokens = 0
total_tokens = 0

# Load the JSONL files
for file in jsonl_files:    
    with open(f"{output_folder}/{file}", "r") as f:
        for line in f:
            item = json.loads(line)            
            item_key = item['custom_id']            
            # grab the matching item in our list            
            list_item = next((x for x in list_data["data"] if x['uniqueId'] == item_key), None)
            if list_item:
              if item["response"]["status_code"] != 200:
                list_item["step_1"] = None
                list_item["metadata"]["step_1"]["error"] = item["response"]
                continue                
              json_res = parse_JSON(item['response']['body']['choices'][0]['message']['tool_calls'][0]['function']['arguments'])
              if 'responder_category' in json_res:
                json_res['responder_category'] = get_correct_category(json_res['responder_category'])
                if list_item['group'] == None:
                  list_item['group'] = json_res['responder_category']
                  list_item['metadata']['aiResponderCategory'] = 'AI'
              list_item["step_1"] = json_res
              list_item["metadata"]["step_1"]["system_fingerprint"] = item['response']['body']['system_fingerprint']
              list_item["metadata"]["step_1"]["batch_id"] = item['id']
              prompt_tokens += item['response']['body']["usage"]["prompt_tokens"]
              completion_tokens += item['response']['body']["usage"]["completion_tokens"]
              total_tokens += item['response']['body']["usage"]["total_tokens"]

local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

list_data["metadata"]["step_1"]["batch_processed"] = {"timestamp": current_time}
list_data["metadata"]["step_1"]["usage"] = {"prompt_tokens": prompt_tokens, "completion_tokens": completion_tokens, "total_tokens": total_tokens}

# Save the updated list back to the json file
with open('./data/step1/list.json', 'w') as f:
    json.dump(list_data, f)

# Export the list to an Excel file for review
# Convert JSON to DataFrame
df = pd.json_normalize(list_data["data"])

# Save DataFrame to Excel
df.to_excel('./data/step1/step1.xlsx', index=False)

### Review AI Classification

This is the last stage of step 1. Here we list and review cases where the AI has
classified a submission into one of our categories of interest (news, academic,
civil, platform or industry) and where we had not originally classified them as
so.

The next two cells do two things:

- **Cell one:** Will list the submission numbers for each category where the AI
  has provided a categorisation of interest
- **Cell two:** For those listed submissions, we can manually update the
  classification as required.


In [ ]:
# Cell 7 (one)
import json
from datetime import datetime
import pytz

# Load original JSON list
with open('./data/step1/list.json', 'r') as f:
    list_data = json.load(f)

# Filter AI-generated items which are grouped and not 'individual'
ai_generated_items = [item for item in list_data["data"] if item['metadata']['groupDefinedBy'] == 'AI' and item['group'] and item['group'] != 'individual']

# Create a dictionary to store the grouped items
grouped_items_dict = {}

# Group the items based on the 'group' value
for item in ai_generated_items:
    group = item['group']
    if group not in grouped_items_dict:
        grouped_items_dict[group] = []
    grouped_items_dict[group].append(item['uniqueId'])

# Print the grouped items
for group, items in grouped_items_dict.items():
    print(f"Group: {group}")
    for item in items:
        print(item)

# Get the current time in the specified timezone
local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

# Update the metadata with the classification check
list_data["metadata"]["step_1"]["classification_check"] = {
    "timestamp": current_time,
    "groups_to_review": grouped_items_dict
}

# Save the updated list back to the json file
with open('./data/step1/list.json', 'w') as f:
    json.dump(list_data, f)


In [6]:
# Cell 8 (two) - manual update with correct classification
import json
from datetime import datetime
import pytz

# The below lists are the items that need to be manually updated with the correct classification
# Add to each list the uniqueId of the item that needs to be updated to the category matching the list (i.e. if submission '30105' should be in the 'academic' category, add '30105' to the 'academic' list)

academic = []
civil = []
platform = []
news = []
individual = []
industry = []

# Load original JSON list
with open('./data/step1/list.json', 'r') as f:
    list_data = json.load(f)

def update_classification(category_list, group_name):
    for item in category_list:
        list_item = next((x for x in list_data if x['uniqueId'] == item), None)
        if list_item:
            list_item['group'] = group_name
            list_item['metadata']['groupDefinedBy'] = 'human'

update_classification(academic, 'academic')
update_classification(civil, 'civil')
update_classification(platform, 'platform')
update_classification(news, 'news')
update_classification(individual, 'individual')
update_classification(industry, 'industry')

local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

list_data["metadata"]["step_1"]["classification_check"]["human_review"] = {
    "timestamp":current_time,
    "fixed": {
        "academic": academic,
        "civil": civil,
        "platform": platform,
        "news": news,
        "individual": individual,
        "industry": industry
        }
    }

# Save the updated list back to the json file
with open('./data/step1/list.json', 'w') as f:
    json.dump(list_data, f)